In [1]:
from flask import Flask, render_template, request, jsonify
import os
import numpy as np
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from langdetect import detect

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
app = Flask(__name__)

In [3]:
# Load model
try:
    with open('naive_bayes_model.pkl', 'rb') as file:
        model_train = pickle.load(file)
except FileNotFoundError:
    print("Error: unable to find the trained model file named 'naive_bayes_model.pkl'.")
    exit()

In [4]:
# Load the vectorizer
try:
    with open('tfidf_vectorizer.pkl', 'rb') as file:
        model_vectorizer = pickle.load(file)
except FileNotFoundError:
    print("Error: unable to find the vectorizer file named 'tfidf_vectorizer.pkl'.")
    exit()

In [5]:
translator = str.maketrans('', '', string.punctuation)

In [6]:
def preprocess_article(article):
    article = article.lower()
    article = article.translate(translator)
    stop_words = stopwords.words('english')
    article = ' '.join([word for word in article.split() if word not in (stop_words)])
    return article

In [7]:
def predict_fake_news_naive_bayes(X, prob_fake, prob_real, fake_word_probs, real_word_probs, alpha=1):
    num_fake = 0
    num_real = 0
    y_pred = []
    for i in range(X.shape[0]):
        article = X[i, :]
        prob_real_article = 1.0
        prob_fake_article = 1.0
        words = article.nonzero()[1]
        for index in words:
            if index in fake_word_probs:
                prob_fake_article *= fake_word_probs[index]
            else:
                prob_fake_article *= alpha / (num_fake + 2 * alpha)
            if index in real_word_probs:
                prob_real_article *= real_word_probs[index]
            else:
                prob_real_article *= alpha / (num_real + 2 * alpha)
        
        
        # Predict the class label of the news article
        if prob_fake_article * prob_fake > prob_real_article * prob_real:
            y_pred.append(1)
        else:
            y_pred.append(0)
            
        if y_pred[-1] == 1:
            num_fake += 1
        else:
            num_real += 1
            
    return y_pred


In [8]:
#Flask routing 
@app.route("/")

def home():
    return render_template("Home.html")

In [9]:
@app.route('/predict', methods = ['POST','GET'])
def predict_news():
    if request.method == 'POST':
        news = request.form['message']
        try: 
            language = detect(news)
            print(language)  # Output: 'en' (English)
            if language == 'en':
                news = preprocess_article(news)
                vectorized_article = model_vectorizer.transform([news])
                
                
                
                prediction = predict_fake_news_naive_bayes(vectorized_article, model_train["prob_fake"], model_train["prob_real"], 
                                    model_train["fake_word_probs"], model_train["real_word_probs"])[0]
                pred = [prediction]
                print(pred)
                return render_template('Home.html', prediction=pred)
            else:
                return render_template('Home.html', error="Provide a valid news")
            
        except Exception as e:
            error = str(e)
            return render_template('Home.html', error=error)
    else:
        return render_template('Home.html')


In [10]:
if __name__ == '__main__':
    app.run(debug = True, host = '0.0.0.0', port='9014')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
